In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dftrain = pd.read_csv('/kaggle/input/cleanee1/LabelEncoded.csv')

In [ ]:
cat_cols = ['Street','City','County','State','Airport_Code','Wind_Direction','Weather_Condition','Sunrise_Sunset']

# Convert categorical columns to category type
for col in cat_cols:
    dftrain[col] = dftrain[col].astype('category')

In [ ]:
for cols in dftrain.columns:
    print(cols, len(dftrain[cols].unique()))

In [ ]:
df1 = dftrain.drop(columns=['Unnamed: 0'])

In [ ]:
df_ = df1.drop(columns=['Severity'])

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re as re
from collections import Counter

from tqdm.auto import tqdm
import math
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import warnings
warnings.filterwarnings('ignore')

import time
from xgboost import XGBClassifier
%matplotlib inline
tqdm.pandas()


In [ ]:
X = df1.drop(columns=['Severity'])
y = df1['Severity']
y = y-np.ones(y.shape[0])

In [ ]:
y,df1['Severity']

In [ ]:
from sklearn.model_selection import train_test_split
X_train_, X_test, y_train_, y_test = train_test_split(X,y , random_state=42,test_size=0.20, shuffle=False)

In [ ]:
FOLDS = 20
SEED = 1004
xgb_models = []
xgb_oof = []
predictions = np.zeros(((y_test.shape[0]),4))
f_imp = []
xgb_valAccu=[]
xgb_TrainAccu=[]

counter = 0
# X = X_train_
# y = y_train_
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    if (fold + 1) % 5 == 0 or (fold + 1) == 1:
        print(f'{"#" * 24} Training FOLD {fold + 1} {"#" * 24}')
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    watchlist = [(X_train, y_train), (X_valid, y_valid)]

    # XGboost model and fit with GPU support
    model = XGBClassifier(n_estimators=1000, n_jobs=-1, max_depth=3, eta=0.2, colsample_bytree=0.67, tree_method='gpu_hist',enable_categorical=True,verbosity=0)
    model.fit(X_train, y_train, eval_set=watchlist, early_stopping_rounds=300,verbose=False)

    val_preds = model.predict_proba(X_valid)
    train_preds = model.predict_proba(X_train)
    val_pred = model.predict(X_valid)
#     val_accu = accuracy_score(y_valid, val_preds)
#     print(val_accu)
    val_score = roc_auc_score(y_valid, val_preds,multi_class='ovr')
    train_score = roc_auc_score(y_train, train_preds,multi_class='ovr')
    best_iter = model.best_iteration
    xgb_valAccu.append(val_score)
    xgb_TrainAccu.append(train_score)
#     idx_pred_target = np.vstack([val_idx, val_preds, y_valid]).T  # shape(len(val_idx), 3)
#     f_imp.append({i: j for i, j in zip(X.columns, model.feature_importances_)})
    print(f'{" " * 20}AUC: {val_score:.5f} {" " * 6}Best Iteration: {best_iter}')

    if val_score > 0.80 and train_score > 0.90:
        test_preds = model.predict_proba(X_test)
        predictions += test_preds
        counter += 1

if(counter!=0):
    predictions /= counter
# mean_val_auc = get_mean_auc(np.array(xgb_oof))
xgb_TestAccu = xgb_valAccu

In [ ]:
print("Mean of Training score")
print(np.mean(xgb_TrainAccu))

In [ ]:
print("Mean of Testing score")
print(np.mean(xgb_TestAccu))